## 3. 데이터 수집 단계
#### 수집 할 내용
#####   ㄱ) 실시간 랭킹 100위
#####   ㄴ) 1시간 단위로 데이터 수집
#####   ㄷ) 실시간 랭킹 100위
######  - 랭킹반영시간, 순위, 제목, 작가, 별점, 전체회차수, 장르, 연령등급, 줄거리, 조회수, 댓글수, 키워드, 연재개시일

#### ※19세 이용가 데이터 있으므로 로그인 및 성인인증 필요

# crawling 변수

## click
#### login_btn__click : 로그인버튼클릭
#### web_novel_click : 웹소설 클릭
####  novel_click :작품 클릭
####  novel_info : 정보 클릭  
####  login : 로그인  
####  go : 로그인 계속하기   
####  cupon : 대여권증정 확인클릭

## 기타
####  ranking_time : 랭킹반영시간 ex : '23.09.27 13:00 기준' 
#### input_id : 아이디창
#### input_pw :패스워드창

## columns
#### ranking : 순위 정보
#### name : 제목 
#### writer : 작가
#### view :조회수
#### star : 별점
#### comment : 댓글 수
#### total :전체 회차
#### start : 연재개시일
#### genre : 분류, 장르
#### age_graded : 연령등급
#### story :줄거리
#### keyword :키워드


In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
driver=webdriver.Chrome()
driver.get("https://page.kakao.com/")
time.sleep(3)

login_btn_click=driver.find_elements("css selector", "#__next > div > \
div.sticky.inset-x-0.top-0.left-0.z-100.flex.w-full.flex-col.items-center.justify-center.bg-bg-a-10 > \
div > div.flex.h-pc_header_height_px.w-1200pxr.items-center.px-30pxr > \
div.mr-16pxr.flex.shrink-0.items-center.justify-end.space-x-24pxr > button.pr-16pxr")[0]

login_btn_click.click()
time.sleep(3)  #로그인클릭

In [3]:
input_id=driver.find_elements("css selector","#loginId--1")[0]#i
input_pw=driver.find_elements("css selector", "#password--2")[0]

input_id.send_keys("  ID 입력  ")   #ID 입력
input_pw.send_keys("  PW 입력  ") #PW 입력

login=driver.find_elements("css selector", "#mainContent > div > div > form > div.confirm_btn > \
button.btn_g.highlight.submit")[0]
login.click()  #로그인
time.sleep(20)  #로그인 인증 해야되니 길게 잡기!

In [19]:
go=driver.find_elements("css selector", "#mArticle > div > div.wrap_btn > form > button")[0]
go.click()  #계속하기
time.sleep(5)

In [20]:
web_novel_click=driver.find_elements("css selector", "#__next > div > div.sticky.inset-x-0.top-0. \
left-0.z-100.flex.w-full.flex-col.items-center.justify-center.bg-bg-a-10 > div >  \
div.flex.h-pc_header_height_px.w-1200pxr.items-center.px-30pxr >  \
div.ml-42pxr.flex-1.items-center.space-x-32pxr > a:nth-child(3)")[0]

web_novel_click.click()
time.sleep(5)  #웹소설 탭 클릭

Real_time_ranking=driver.find_elements("css selector", "#tab-depth-2 > div > div > \
ul > li:nth-child(3) > a")[0]

Real_time_ranking.click()
time.sleep(7) #실시간랭킹 클릭

In [31]:
def data_crawling() :
    driver.refresh()  #시간대가 변경으로 새로고침 후 진행해야함
    time.sleep(15)
    
    kakao_top1_100=[]

    for i in range(1,101) : #1~70 / 71~100 나누어서 진행
        if i>50 :  # i가 50이 넘으면 스크롤을 내리고 10초 대기
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")  
            time.sleep(10) #i가 50이 넘으면 스크롤을 내리고 10초 대기한다.


        html=driver.page_source
        soup=BeautifulSoup(html, 'html.parser') #페이지소스 가져오기 
        
        ranking=soup.select(
            f"div > div:nth-child({i}) > div > a > div > div.jsx-2079337112.h-76pxr. \
            w-full.space-y-4pxr.pt-8pxr.pr-8pxr.pb-4pxr > div.under-320-view\:w-56pxr. \
            flex.w-68pxr.rounded-\[5px\].px-4pxr.bg-el-20 > div.font-small2-bold.flex-1. \
            text-el-60")[0].text
            #ranking : 순위 정보
            
        ranking_time=soup.select("#__next > div > div.flex.w-full.grow.flex-col.px-122pxr > \
            div > div.flex.grow.flex-col > div:nth-child(3) > div > div.bg-bg-a-10 > div > \
            div > span")[0].text
            # ranking_time : 기준시간  
            # 기준시간은 매일 새롭게 직접 업데이트해야함

        try :  

            novel_click=driver.find_elements("css selector", f"div > div:nth-child({i}) > \
            div > a > div > div.jsx-2079337112.relative")[0]
            
            novel_click.click() #순차적으로 작품 클릭
            time.sleep(3)

            driver.refresh()    #대여권이 리셋을 위해 새로고침
            time.sleep(6)

            html=driver.page_source  #페이지소스 가져오기
            soup=BeautifulSoup(html, 'html.parser') 

            name=soup.select(" a > div > span.font-large2-bold.mb-3pxr.text-ellipsis. \
            break-all.text-el-70.line-clamp-2")[0].text
                # name : 제목 

            writer=soup.select(" a > div > span.font-small2.mb-6pxr.text-ellipsis. \
            text-left.text-el-70.opacity-70.break-word-anywhere.line-clamp-2")[0].text
                # writer : 작가    
            
            view=soup.select("a > div > div.flex.h-16pxr.items-center.justify-center. \
            all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > \
            div:nth-child(2) > span")[0].text    
                # view :조회수   
                
            star=soup.select("a > div > div.flex.h-16pxr.items-center.justify-center. \
            all-child\:font-small2.\[\&\>\*\:not\(\:last-child\)\]\:mr-10pxr > \
            div:nth-child(3) > span")[0].text
               # star : 별점    
                
            comment=soup.select("div.flex-1.flex.flex-col > div.mt-4pxr.flex.min-h-200pxr. \
            flex-1.flex-col.overflow-hidden.rounded-12pxr.bg-bg-a-20 > div > div.flex. \
            h-44pxr.w-full.flex-row.items-center.justify-between.px-18pxr.bg-bg-a-20 > \
            div.flex.h-full.flex-1.items-center.space-x-8pxr > span")[0].text[3:]
               # comment : 댓글 수       
                
            total=soup.select("div.flex-1.flex.flex-col > div.rounded-b-12pxr.bg-bg-a-20 > \
            div.flex.h-44pxr.w-full.flex-row.items-center.justify-between.bg-bg-a-20.px-18pxr > \
            div.flex.h-full.flex-1.items-center.space-x-8pxr > span")[0].text[3:]
                #total :전체 회차    
                
            start=soup.select("div > div.flex.flex-col.pr-14pxr > div.text-ellipsis. \
            line-clamp-1.font-x-small1.h-16pxr.text-el-50 > span")[0].text
                #start : 연재개시일


            novel_info=driver.find_elements("css selector", "#__next > div > \
            div.flex.w-full.grow.flex-col.px-122pxr > div.flex.h-full.flex-1 > \
            div.mb-28pxr.ml-4px.flex.w-632pxr.flex-col.overflow-hidden.rounded-12pxr > \
            div.relative.flex.w-full.flex-col.my-0.bg-bg-a-20.px-15pxr.pt-28pxr.pb-12pxr > \
            div > div > div:nth-child(2) > a")[0]
            
            novel_info.click() # novel_info : 정보탭 클릭  
            time.sleep(8)   


            html=driver.page_source
            soup=BeautifulSoup(html, 'html.parser') #페이지소스 가져오기

            genre=soup.select("div > div:nth-child(2) > div > span:nth-child(3)")[0].text
                # genre : 분류, 장르
                
            age_graded=soup.select("div > div:nth-child(4) > span.text-el-70. \
            break-word-anywhere")[0].text
                # age_graded : 연령등급
                
            story=soup.select("div > div.relative.w-full.overflow-hidden.max-h-\[144px\] > \
            span")[0].text
                # story :줄거리 / 나중에 데이터 만들고 \n\n 수정할지 결정하기
                
            keyword=soup.select("div:nth-child(3) > div.flex.w-full.flex-col.items-center.\
            overflow-hidden > div")[0].text
                # keyword :키워드    

    #         driver.back()          #뒤로가기 2번 or 입력한 주소 실행 1번...이 에러가 적음
    #         driver.back()
            driver.get("https://page.kakao.com/menu/10011/screen/94")
            time.sleep(15)   #잘 실행되는지 확인


        except IndexError as e :       #키워드가 없어서 에러나면 
            keyword="NaN"               #키워드는 "NaN"
    #         driver.back()             #뒤로가기 2번 or 입력한 주소 실행 1번...이 에러가 적음
    #         driver.back()
            driver.get("https://page.kakao.com/menu/10011/screen/94")
            time.sleep(15)   #잘 실행되는지 확인용


        kakao_top1_100.append([ranking_time,ranking,name,writer,view,star,comment,total,start,genre,age_graded,story,keyword])            
        #리스트에 담고            
            
    kakao_df=pd.DataFrame(data=kakao_top1_100, index=range(1,101), columns=['기준시간','순위','제목','작가','조회수','별점','댓글수','전체회차수','연재개시일','장르','연령등급','줄거리','키워드'])
    #데이터프레임 만들고                 
        
    file_name=ranking_time[:-6]+" 시_kakao_top1_100.xlsx"    
     #파일명 현지시간+순위
        
    kakao_df.to_excel(file_name, index=False)         
    #데이터프레임을 엑셀로 저장(파일명, 인덱스없음)

※ 1시간마다 반복 실행하도록 자동화 코드 짰지만, 
jupyter notebook에서 무한반복되는 이슈로 
시간 별 time.sleep으로 진행※

In [ ]:
data_crawling() #00:20

In [21]:
time.sleep(2220) #01:20
data_crawling()

In [ ]:
time.sleep(2220) #02:20
data_crawling()

In [ ]:
time.sleep(2220) #03:20
data_crawling()

In [ ]:
time.sleep(2220) #04:20
data_crawling()

In [ ]:
time.sleep(2220) #05:20
data_crawling()

In [ ]:
time.sleep(2220) #06:20
data_crawling()

In [ ]:
time.sleep(2220) #07:20
data_crawling()

In [ ]:
time.sleep(2220) #08:20
data_crawling()

In [ ]:
time.sleep(2220) #09:20
data_crawling()

In [ ]:
time.sleep(2220) #10:20    top71~100 error로 데이터 수집 못함
data_crawling()

In [ ]:
data_crawling()  #11:20  

In [ ]:
time.sleep(2220) #12:20
data_crawling()

In [ ]:
time.sleep(2220) #13:20
data_crawling()

In [ ]:
time.sleep(2220) #14:20
data_crawling()

In [ ]:
time.sleep(2220) #15:20
data_crawling()

In [ ]:
time.sleep(2220) #16:20
data_crawling()

In [ ]:
time.sleep(2220) #17:20
data_crawling()

In [ ]:
time.sleep(2220) #18:20
data_crawling()

In [ ]:
time.sleep(2220) #19:20
data_crawling()

In [ ]:
time.sleep(2220) #20:20
data_crawling()

In [ ]:
time.sleep(2220) #21:20
data_crawling()

In [ ]:
time.sleep(2220) #22:20
data_crawling()

In [ ]:
time.sleep(2220) #23:20
data_crawling()

In [ ]:
time.sleep(2220) #24:20
data_crawling()